In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim
import numpy as np
import plotly.graph_objs as go
import pandas
import os
import psutil #to get the memory used
import os


In [17]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(device)

cuda


In [ ]:
combined_sequences = []

leave = []  # List to store indices of files not considered
####change foolder location
dirs = os.listdir('NMP')
c = 0
for d in dirs:
    count = len(list(os.listdir(os.path.join('NMP', d))))
    print(count)
    for i in range(count):
        c += 1
        # Load the .npz file
        ####change foolder location before NMP
        data = np.load(f'NMP/{d}/{d}_{i}.npz')
        
        # Extract relevant data
        position_data = data['position']  # Assuming 'position' is a key in the .npz file
        
        # Extract x, y, z coordinates
        x = position_data[:, 0][:100]  # Get the first 100 x coordinates
        y = position_data[:, 1][:100]  # Get the first 100 y coordinates
        z = position_data[:, 2][:100]  # Get the first 100 z coordinates
        
        # Check if the sequence is long enough
        if len(x) == 100:
            combined_sequence = np.vstack((x, y, z)).T  # Stack x, y, z and transpose to get shape (100, 3)
            combined_sequences.append(combined_sequence)
             ####change foolder location before NMP
            print(f'NMP/trimmed_Bamboo/trimmed_Bamboo_{i}.npz - Loaded successfully')
        else:
             ####change foolder location before NMP
            leave.append(f'NMP/{d}/{d}_{i}.npz')
            print(f'NMP/trimmed_Bamboo/trimmed_Bamboo_{i}.npz - left')
        
        print("---------------------------")


    # Convert list to NumPy array
    combined_array = np.array(combined_sequences)

    # Convert NumPy array to PyTorch tensor
    combined_tensor = torch.tensor(combined_array, dtype=torch.float32)


    # Now combined_tensor can be used for training your LSTM model
print(combined_tensor.shape)  # Should print torch.Size([163, 100, 3])

print(c)

In [19]:
combined_tensor = combined_tensor.to(device)

In [20]:

test_data = np.load(f'NMP/trimmed_Bamboo/trimmed_Bamboo_{47}.npz')

position_data = test_data['position'][:100]  # Limit to the first 100 samples if necessary

mid_point = position_data.shape[0] // 2

# Prepare the input and target tensors
test_input = torch.tensor(position_data[:mid_point], dtype=torch.float32).unsqueeze(0)  # First half
test_target = torch.tensor(position_data[mid_point:], dtype=torch.float32).unsqueeze(0)  # Second half

# Transfer tensors to the appropriate device (GPU or CPU)
test_input_4 = test_input.to(device)
test_target_4 = test_target.to(device)


In [21]:
position_data = torch.tensor(position_data)
position_data = position_data.to(device)

In [22]:
combined_tensor

tensor([[[ 1.0006,  1.2619,  1.8557],
         [ 0.9993,  1.3053,  1.8195],
         [ 0.9999,  1.3470,  1.7853],
         ...,
         [ 0.8238,  2.0998, -1.2494],
         [ 0.8214,  2.0753, -1.2787],
         [ 0.8192,  2.0507, -1.3079]],

        [[ 1.0979,  1.2421,  1.8938],
         [ 1.0934,  1.2864,  1.8607],
         [ 1.0895,  1.3295,  1.8273],
         ...,
         [ 0.8766,  2.2128, -1.1789],
         [ 0.8738,  2.1914, -1.2095],
         [ 0.8710,  2.1696, -1.2403]],

        [[ 0.9545,  0.7337, -2.0168],
         [ 0.9555,  0.7594, -1.9725],
         [ 0.9570,  0.7893, -1.9274],
         ...,
         [ 1.1758,  1.4907,  1.7906],
         [ 1.1789,  1.4662,  1.8246],
         [ 1.1822,  1.4405,  1.8593]],

        ...,

        [[ 1.3530,  0.6545,  3.4689],
         [ 1.3512,  0.6912,  3.4029],
         [ 1.3490,  0.7340,  3.3395],
         ...,
         [ 0.9526,  2.3103, -1.0961],
         [ 0.9489,  2.2920, -1.1348],
         [ 0.9454,  2.2731, -1.1734]],

        [[

In [10]:
class TransformerModel(nn.Module):
    def __init__(self, n_input=3, n_hidden=128, n_output=3, n_layers=3, n_heads=8, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.n_hidden = n_hidden
        self.embedding = nn.Linear(n_input, n_hidden)
        self.positional_encoding = PositionalEncoding(n_hidden, dropout)
        
        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=n_hidden, nhead=n_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        
        # Decoder to map to output dimensions
        self.decoder = nn.Linear(n_hidden, n_output)

    # In the forward method of your Transformer model
    def forward(self, src, future=0):
        src = self.embedding(src) * torch.sqrt(torch.tensor(self.n_hidden, dtype=torch.float32))
        src = self.positional_encoding(src)
        
        # Permute to [sequence_length, batch_size, embedding_size] if necessary
        src = src.permute(1, 0, 2)  # Shape adjustment for transformer input
        
        # Pass through the transformer encoder
        output = self.transformer_encoder(src)
        
        # Permute back to original [batch_size, sequence_length, embedding_size]
        output = output.permute(1, 0, 2)
        output = self.decoder(output)
        
        return output


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [ ]:
class lstmpred(nn.Module):
    def __init__(self, n_hidden=51):
        super(lstmpred, self).__init__()
        self.n_hidden = n_hidden
        self.lstm1 = nn.LSTMCell(3, self.n_hidden)
        self.lstm2 = nn.LSTMCell(self.n_hidden, self.n_hidden)
        self.lstm3 = nn.LSTMCell(self.n_hidden, self.n_hidden)  # lstm3
        self.lstm4 = nn.LSTMCell(self.n_hidden, self.n_hidden)  # lstm4
        self.lstm5 = nn.LSTMCell(self.n_hidden, self.n_hidden)  # lstm5 (new layer)
        self.lstm6 = nn.LSTMCell(self.n_hidden, self.n_hidden)  # lstm6 (new layer)
        self.linear = nn.Linear(self.n_hidden, 3)

    def forward(self, x, future=0):
        outputs = []
        n_samples = x.size(0)  # Batch size

        # Initialize hidden and cell states for all LSTM layers
        h_t = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        c_t = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        h_t2 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        c_t2 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        h_t3 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        c_t3 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        h_t4 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        c_t4 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        #h_t5 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)  # For lstm5
        #c_t5 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)  # For lstm5
        #h_t6 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)  # For lstm6
        #c_t6 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)  # For lstm6

        for input in x.split(1, dim=1):  # Input shape [batch_size, 1, 3]
            input = input.squeeze(1)  # Shape [batch_size, 3]
            h_t, c_t = self.lstm1(input, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            h_t3, c_t3 = self.lstm3(h_t2, (h_t3, c_t3))
            h_t4, c_t4 = self.lstm4(h_t3, (h_t4, c_t4))
            #h_t5, c_t5 = self.lstm5(h_t4, (h_t5, c_t5))  # Add lstm5 layer processing
            #h_t6, c_t6 = self.lstm6(h_t5, (h_t6, c_t6))  # Add lstm6 layer processing
            output = self.linear(h_t4)  # Output from the last LSTM layer (lstm6)
            outputs.append(output)

        for i in range(future):
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            h_t3, c_t3 = self.lstm3(h_t2, (h_t3, c_t3))
            h_t4, c_t4 = self.lstm4(h_t3, (h_t4, c_t4))
            #h_t5, c_t5 = self.lstm5(h_t4, (h_t5, c_t5))  # Add lstm5 layer processing
            #h_t6, c_t6 = self.lstm6(h_t5, (h_t6, c_t6))  # Add lstm6 layer processing
            output = self.linear(h_t4)
            outputs.append(output)

        outputs = torch.stack(outputs, dim=1)  # Shape [batch_size, sequence_length, 3]
        return outputs

In [12]:
train_input = combined_tensor[3:, :-1]   # leave the last time step for training input
train_target = combined_tensor[3:, 1:]   # training target is shifted by one step
test_input = combined_tensor[:3, :-1]    # first 3 sequences for testing
test_target = combined_tensor[:3, 1:]    # testing target is also shifted by one step

train_input = train_input.permute(1, 0, 2)  # Adjust for transformer input
train_target = train_target.permute(1, 0, 2)  # Ensure targets match shape as needed

In [ ]:
###this is for  overall trajectory optimisation
# Initialize the model
model = TransformerModel()  
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.LBFGS(model.parameters(), lr=0.8)
n_steps = 100  # Number of steps to train the model

for i in range(n_steps):
    print("Step:", i + 1)

    def closure():
        optimizer.zero_grad()
        out = model(train_input)
        loss = criterion(out, train_target)
        print('Training loss:', loss.item())
        loss.backward()
        return loss

    optimizer.step(closure)
    pid = os.getpid()
    py = psutil.Process(pid)
    memoryUse = py.memory_info()[0] / 2.**30  # Convert from bytes to GB
    print(f"Memory Usage: {memoryUse:.2f} GB")

    # Testing phase
    with torch.no_grad():
        future = 100  # Predict 100 future steps beyond the test sequence
        pred = model(test_input, future=future)
        loss = criterion(pred[:, :-future], test_target)
        print('Test loss:', loss.item())
        y_pred = pred.cpu().detach().numpy()

In [ ]:
torch.save(model.state_dict(), 'model-transformer.pth')

In [23]:

model = torch.load('model.pt')
model.eval()
with torch.no_grad():
    # Predict the second half of the sequence based on the first half
    future_steps = test_target_4.size(1)  # Number of future steps equals the length of the second half
    prediction = model(test_input_4, future=future_steps)

    # Extract the predicted second half
    predicted_output = prediction[:, -future_steps:]  # Only take the predicted future steps

    # Move the tensors to CPU and convert to NumPy arrays
    prediction = prediction.cpu().detach().numpy()
    predicted_output = predicted_output.cpu().detach().numpy()

In [24]:
position_data = position_data.cpu().numpy()

In [25]:
prediction.squeeze(0)

array([[ 1.2757837 ,  1.2094941 ,  3.074891  ],
       [ 1.2795182 ,  1.2619383 ,  3.0293748 ],
       [ 1.2568407 ,  1.3041964 ,  2.9469788 ],
       [ 1.2640147 ,  1.3264451 ,  2.8960125 ],
       [ 1.2619343 ,  1.3565564 ,  2.832408  ],
       [ 1.2572932 ,  1.3892791 ,  2.771588  ],
       [ 1.251484  ,  1.420764  ,  2.7119644 ],
       [ 1.245144  ,  1.4503865 ,  2.652693  ],
       [ 1.2388198 ,  1.47839   ,  2.5922272 ],
       [ 1.2327343 ,  1.5059452 ,  2.5319366 ],
       [ 1.2293607 ,  1.530251  ,  2.4739208 ],
       [ 1.2229029 ,  1.5569589 ,  2.4113827 ],
       [ 1.2179047 ,  1.5809803 ,  2.3520353 ],
       [ 1.2111878 ,  1.606188  ,  2.2900043 ],
       [ 1.2057424 ,  1.6288772 ,  2.2310658 ],
       [ 1.2007631 ,  1.6511042 ,  2.1711264 ],
       [ 1.1959758 ,  1.672478  ,  2.110871  ],
       [ 1.1904755 ,  1.6935959 ,  2.049903  ],
       [ 1.1852107 ,  1.714423  ,  1.9893123 ],
       [ 1.1810306 ,  1.7352836 ,  1.9306344 ],
       [ 1.176589  ,  1.7543468 ,  1.870

In [26]:
prediction = np.squeeze(prediction, axis=0)

In [27]:
prediction.shape

(100, 3)

In [28]:
# Extract x, y, z coordinates from the predictions
predicted_x = prediction[:, 0]
predicted_y = prediction[:, 1]
predicted_z = prediction[:, 2]


actual_x = position_data[:, 0]
actual_y = position_data[:, 1]
actual_z = position_data[:, 2]

# Create the 3D line plot for the actual data (first half)
trace_actual = go.Scatter3d(
    x=actual_x,
    y=actual_y,
    z=actual_z,
    mode='lines+markers',  # Both lines and markers
    marker=dict(size=5, color='blue'),  # Marker size and color for actual data
    line=dict(color='blue', width=2),  # Line color and width for actual data
    name='Actual Data (First Half)'
)

# Create the 3D line plot for the predicted data (second half)
trace_predicted = go.Scatter3d(
    x=predicted_x,
    y=predicted_y,
    z=predicted_z,
    mode='lines+markers',  # Both lines and markers
    marker=dict(size=5, color='red'),  # Marker size and color for predicted data
    line=dict(color='red', width=2),  # Line color and width for predicted data
    name='Predicted Data (Second Half)'
)

# Create layout
layout = go.Layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title="Interactive 3D Trajectory: Actual (First Half) and Predicted (Second Half)"
)

# Create the figure with both traces and display it
fig = go.Figure(data=[trace_actual, trace_predicted], layout=layout)
fig.show()